## SECTION 1 
### Introduction to the problem/task and dataset

PumpkinSeed-ML-Insights is a comprehensive repository dedicated to the exploration and analysis of Turkish pumpkin seed varieties, with a focus on classifying whether a seed belongs to Urgup Sivrisi or Cercevelik species. This project demonstrates the knowledge of authors in data science and machine learning.

Within this repository, you will find a Jupyter Notebook that serves as a self-explanatory document, guiding you through the entire process. This repository also contains three Python files, each implementing a different machine learning model: `knn_pumpkinseed.py`, `logistic_regression_pumkinseed.py`, and `neural_network_pumpkinseed.py`. There is also the `pumpkin_seeds.csv`, which contains the data and `Pumpkin_seeds.pdf`, which contains some description of the dataset.



---
## SECTION 2
### Description of the dataset

`pumpkin_seeds.csv` is a CSV file containing information about Pumpkin Seeds found in Turkey. 

This dataset came from the study `The use of machine learning methods in classification of pumpkin seeds (Cucurbita pepo L.).` by Koklu, M., Sarigil, S., & Ozbek, O. in 2021. In their paper, they used a product shooting box to obtain quality images of the pumpkin seeds. The authors converted the images to a gray tone and then to binary images. To convert the image data into a CSV file, they extracted 12 morphological features.

Overall, the CSV file has 13 columns/features and 2500 rows. The first 12 columns are from the extracted morphological features, while the last column classifies whether it belongs to the Urgup Sivrisi or Cercevelik species. There are 2500 rows, representing a single seed used in the study. There are 1200 Urgup Sivrisi and 1300 Cercevelik species of pumpkin seeds. 

The features found in this CSV file are as follows:
1. Area – Number of pixels within the borders of a pumpkin seed
2. Perimeter – Circumference in pixels of a pumpkin seed
3. Major_Axis_Length – Large axis distance of a pumpkin seed
4. Minor_Axis_Length – Small axis distance of a pumpkin seed
5. Convex_Area – Number of pixels of the smallest convex shell at the region formed by the
pumpkin seed.
6. Equiv_Diameter – Computed as !4𝑎⁄𝜋, where 𝑎 is the area of the pumpkin seed.
7. Eccentricity – Eccentricity of a pumpkin seed
8. Solidity – Convex condition of the pumpkin seeds
9. Extent – Ratio of a pumpkin seed area to the bounding box pixels
10. Roundness – Ovality of pumpkin seeds without considering the distortion of the edges.
11. Aspect_Ration – Aspect ratio of the pumpkin seeds
12. Compactness – Proportion of the area of the pumpkin seed relative to the area of the circle
with the same circumference
13. Class – Either Cercevelik or Urgup Sivrisi



---
## SECTION 3

### List of Requirements

---
## SECTION 4

### Data preprocessing and cleaning



In [ ]:
import numpy as np
import pandas as pd
import csv


In [ ]:
data = []

with open('pumpkin_seeds.csv', 'r', encoding='utf-8', errors='replace') as csv_file:
    raw_data = csv.reader(csv_file)

    #Skip headers
    next(raw_data)

    #Store data into data array
    for row in raw_data:
        row_data = []
        for i in range(13): #Convert errors into 1 or 2 (depending on their specie)
            if i == 12 and row[i] == '�er�evelik':
                row_data.append(int(0))
            elif i == 12 and row[i] == '�rg�p Sivrisi':
                row_data.append(int(1))
            else:
                row_data.append(row[i])

        data.append(row_data)

#Convert data into numpy array
np_data = np.array(data)
np_data



The first step is to address the encoding issues, especially in the "Class" column. The unique values in the "Class" column are showing encoding issues, as evidenced by the presence of escape characters like \x82. These values are intended to represent the two species of pumpkin seeds. To correct this, we need to replace these incorrectly encoded strings with a correct format. We replaced the string class names with an integer value of 0 and 1.

### Data Scaling

The numerical features are scaled using `StandardScaler` from `sklearn.preprocessing`. This ensures that all features have a mean of 0 and a standard deviation of 1, which is particularly important for many machine learning algorithms.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Replace 'original_column_names' with the actual list of column names
original_column_names = ['area', 'perimeter', 'major_axis_length', 'minor_axis_length',
                         'convex_area', 'equiv_diameter', 'eccentricity', 'solidity',
                         'extent', 'roundness', 'aspect_Ration', 'compactness', 'class']

# Convert the numpy array to a pandas DataFrame using the original column names
pumpkin_seeds_data = pd.DataFrame(np_data, columns=original_column_names)

# Selecting only the numerical features for scaling
numerical_features = pumpkin_seeds_data.iloc[:, :-1]

# Initializing the Standard Scaler
scaler = StandardScaler()

# Scaling the numerical features
scaled_numerical_features = scaler.fit_transform(numerical_features)

# Creating a new DataFrame with scaled values using the original column names (excluding 'Class')
scaled_numerical_df = pd.DataFrame(scaled_numerical_features, columns=original_column_names[:-1])

# Adding the non-numerical column ('Class') back to the DataFrame
scaled_pumpkin_seeds_data = pd.concat([scaled_numerical_df, pumpkin_seeds_data['class']], axis=1)

scaled_pumpkin_seeds_data

# Creating the feature set 'X' and the target 'y'
X = scaled_pumpkin_seeds_data.iloc[:, :-1]  # All columns except the last one
y = scaled_pumpkin_seeds_data['class']      # Only the last column

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)




---------------------------------------------------------------------------
## SECTION 5 - Exploratory Data Analysis (EDA)

---------------------------------------------------------------------------
## SECTION 6 - Model training

### kNN

In [ ]:
print(np_data.shape)
print("testbryce")


### Logistic Regression

### Naive Bayes

The third model we will be evaluating is the Gaussian Naive Bayes model. This model is based on the Bayes theorem, which describes the probability of an event based on prior knowledge of conditions that might be related to the event. The Gaussian Naive Bayes model assumes that the features are normally distributed, which is generally true for our dataset as seen in our EDA. We chose the Gaussian variant over the multinomial variant because our dataset is continuous and not discrete. The multinomial variant is used for discrete features, such as word counts for text classification.

In [ ]:
from sklearn.naive_bayes import GaussianNB
seed_nb = GaussianNB()
seed_nb.fit(X_train, y_train)
print(seed_nb.class_count_)
print(seed_nb.class_prior_)


The GNB model is implemented using the `GaussianNB` class from `sklearn.naive_bayes`. `GaussianNB()` has two parameters: `priors` and `var_smoothing`. `priors` is an array of prior probabilities of the classes. If it is not specified, the prior probabilities are automatically adjusted according to the data. `var_smoothing` is a smoothing parameter that is used to compute the variance of the features. The default value is `1e-9`.

In [ ]:
predictions = seed_nb.predict(X_train)

def compute_accuracy(predictions, actual):
    return (np.sum(predictions == actual)/len(actual)) * 100

print("Training accuracy: ", compute_accuracy(predictions, y_train), "%")

predictions = seed_nb.predict(X_test)
predictions
print("Test accuracy: ", compute_accuracy(predictions, y_test), "%")


As you can see, the GNB model has a training accuracy of 87.36% and a test accuracy of 86.08%, which are relatively close to each other. This is a sign that the model is not overfitting nor underfitting.

#### Hyperparameter tuning using ParameterGrid

In [ ]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

print("X_test shape : ", X_test.shape)
print("y_test shape : ", y_test.shape)

X_train, X_validation, y_train, y_validation = train_test_split(X_train_val, y_train_val, test_size=0.25, stratify=y_train_val, random_state=42)

print("X_train shape : ", X_train.shape)
print("y_train shape : ", y_train.shape)
print("X_validation shape : ", X_validation.shape)
print("y_validation shape : ", y_validation.shape)


In [ ]:
from sklearn.model_selection import ParameterGrid
seed_nb = GaussianNB()
hyperparameters = [{
    'var_smoothing' : np.logspace(-11, -5, num=100),
}]


In [ ]:
best_score = 0
for g in ParameterGrid(hyperparameters):
    print(g)

    seed_nb.set_params(**g)

    # write code here
    seed_nb.fit(X_train, y_train)
    predictions = seed_nb.predict(X_train)
    train_acc = compute_accuracy(predictions, y_train)

    # write code here
    predictions = seed_nb.predict(X_validation)
    val_acc = compute_accuracy(predictions, y_validation)

    print(f"Train acc: {train_acc}% \t Val acc: {val_acc}%", end="\n\n")

    if val_acc > best_score:
        best_score = val_acc
        best_grid = g

print("Best accuracy: ", best_score, "%")
print("Best grid: ", best_grid)


In [ ]:
seed_nb = GaussianNB(**best_grid)
print(seed_nb.get_params())
seed_nb.fit(X_train_val, y_train_val)

predictions = seed_nb.predict(X_test)
test_acc = compute_accuracy(predictions, y_test)
print("Test accuracy: ", test_acc, "%")


In [ ]:
print(seed_nb.class_count_)
print(seed_nb.class_prior_)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)


In [ ]:
seed_nb.fit(X_train, y_train)
predictions = seed_nb.predict(X_test)

test_acc = compute_accuracy(predictions, y_test)
print("Test accuracy: ", test_acc, "%")
